# Wine Quality Clasification Notebook

## Dataset Description
This dataset consists of 1,000 wine samples, where each sample has been assessed using four chemical features and assigned a quality label (low, medium, or high). The dataset is intended for classification tasks—the objective is to build a model that can predict the quality_label of a wine based on its chemical attributes.

Wines can have their quality influenced by various physicochemical factors (e.g., sugar content, acidity, alcohol). Understanding these can help producers and researchers assess wine characteristics and potentially improve production quality.

## Data Collection
The dataset is hosted on Kaggle by user sahideseker and described simply as containing 1,000 labeled wine samples with key attributes. 
[Wine Quality Classification](https://www.kaggle.com/datasets/sahideseker/wine-quality-classification?fbclid=IwY2xjawKri85leHRuA2FlbQIxMABicmlkETFwSmpnYkdhVDFlTnB5Nm9lAR5XrBjEEytHpU0oDCEo4Fs1W97qrXJQrLufftsi_Ro4KXY7ZrGxk17P3rU7Ug_aem_HhBigX5iAqh3X31BoKJfeA)


The structure closely matches the [UCI Wine Quality dataset](https://archive.ics.uci.edu/dataset/186/wine%2Bquality?utm_source=chatgpt.com), which derives from lab-based physicochemical testing of Portuguese Vinho Verde wines, along with sensory (expert) quality scores 
archive.ics.uci.edu


Likely a sample or curated excerpt from the UCI dataset—though no explicit mention is made of whether the data are from red or white varieties, or how the quality labels were assigned.

### Potential Implications of Data Collection
* Label Origin Ambiguity: If labels derive from expert scores versus rule-based thresholds, this can affect model interpretability and reliability.
* Sampling Bias: Without clear provenance, it's unclear whether all wine types or regions are equally represented.
* Limited Feature Scope: Only four chemical attributes are included, so models trained on this may miss nuance present in broader feature sets.



## Structure of the Data
This is a structured, tabular dataset with:
* Rows: Each row corresponds to one wine sample.
* Columns: Each column is either a measurable physicochemical feature or the class label.
* Total Observations: 1000 rows (wine samples)
* Total Features: 5 columns (4 features + 1 label)

## Column-by-Column Breakdown
| Column Name      | Data Type            | Description                                                                                                                     |
| ---------------- | -------------------- | ------------------------------------------------------------------------------------------------------------------------------- |
| `fixed_acidity`  | Float                | Refers to acids that do not evaporate readily. Affects flavor and stability. Higher values often mean higher tartness.          |
| `residual_sugar` | Float                | The amount of sugar left after fermentation. Impacts sweetness.                                                                 |
| `alcohol`        | Float                | Alcohol percentage by volume. Higher alcohol often contributes to a better quality perception.                                  |
| `density`        | Float                | Density of the wine, related to sugar and alcohol content. Important for quality and fermentation monitoring.                   |
| `quality_label`  | Object (Categorical) | The assigned quality class for each wine: `low`, `medium`, or `high`. This is the **target variable** for classification tasks. |





## Data Cleaning

#### Load the data

In [1]:
import pandas as pd

df = pd.read_csv('wine_quality_classification.csv')
df.head()

,fixed_acidity,residual_sugar,alcohol,density,quality_label
0,9.3,6.4,13.6,1.0005,high
1,11.2,2.0,14.0,0.9912,medium
2,11.6,0.9,8.2,0.9935,low
3,12.9,6.6,12.7,1.0002,low
4,13.9,13.8,10.4,0.9942,medium


#### Check for Incorrect Data Types

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fixed_acidity   1000 non-null   float64
 1   residual_sugar  1000 non-null   float64
 2   alcohol         1000 non-null   float64
 3   density         1000 non-null   float64
 4   quality_label   1000 non-null   object 
dtypes: float64(4), object(1)
memory usage: 39.2+ KB


#### Check for Missing Data

In [3]:
df.isnull().sum()

fixed_acidity     0
residual_sugar    0
alcohol           0
density           0
quality_label     0
dtype: int64

#### Check Duplicate Data

In [4]:
df.duplicated().sum()

np.int64(0)

#### Check for Inconsistent Formatting

In [5]:
df.columns = df.columns.str.strip()
print(df.columns.tolist())

['fixed_acidity', 'residual_sugar', 'alcohol', 'density', 'quality_label']


#### Check for Outliers

In [6]:
for col in ['fixed_acidity', 'residual_sugar', 'alcohol', 'density']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)]
    print(f"{col}: {len(outliers)} outliers")


fixed_acidity: 0 outliers
residual_sugar: 0 outliers
alcohol: 0 outliers
density: 0 outliers


## Data Cleaning Summary

After thorough inspection, no errors, inconsistencies, or missing values were found in the dataset.
All columns have appropriate data types, categorical labels are consistently formatted, and no duplicate or null values exist.
The dataset is clean and does not require additional preprocessing.

## Exploratory Data Analysis (EDA)

In this section, we will perform Exploratory Data Analysis (EDA) to better understand the distribution of features, relationships between variables, and patterns associated with wine quality labels. Visualizations and summary statistics will help uncover trends and inform subsequent modeling steps.

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Research Question

**Main Research Question:** "What combination of chemical properties best predicts wine quality classification, and how do these properties interact with each other?"

To explore this main question, we'll investigate the following exploratory data analysis questions:

1. How do the distributions of chemical properties differ across wine quality categories?
2. What relationships exist between the chemical properties of wine, and do these relationships change based on quality level?
3. Are there distinct clusters or patterns in the chemical composition that correspond to quality classifications?

In [8]:
print("Basic statistics of the dataset:")
print(df.describe())

# Distribution of wine quality labels
print("\nDistribution of wine quality labels:")
quality_counts = df['quality_label'].value_counts()
print(quality_counts)

Basic statistics of the dataset:
       fixed_acidity  residual_sugar      alcohol      density
count    1000.000000      1000.00000  1000.000000  1000.000000
mean       10.174300         7.69200    11.039900     0.997505
std         3.452693         4.20892     1.760283     0.004300
min         4.000000         0.50000     8.000000     0.990000
25%         7.100000         4.10000     9.500000     0.993900
50%        10.400000         7.55000    11.000000     0.997400
75%        13.125000        11.50000    12.600000     1.001200
max        16.000000        15.00000    14.000000     1.005000

Distribution of wine quality labels:
quality_label
medium    355
high      343
low       302
Name: count, dtype: int64
